## Flow in this tutorial.
1. Import Lib
2. Load Dataset. `(train.json & test.json)`
3. Data Pre-processing.
4. BERT Embeddin.
5. Build SVM classifier.

## 1. Import Library


In [ ]:
from google.colab import files
import json
import re

import pandas as pd
import numpy as np
from pandas.core.frame import DataFrame
from sklearn.utils import shuffle

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics

You will have to install `transformers` library each time you reconnect.

In [ ]:
!pip install transformers 
from transformers import BertTokenizer, BertModel,BertConfig

     |████████████████████████████████| 1.8MB 19.8MB/s 
     |████████████████████████████████| 2.9MB 58.1MB/s 
     |████████████████████████████████| 890kB 49.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e020b9532d7749be16f5158ef32db727eb61cfe6f188dfbbd32ac3c150579299
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


##2. Load Dataset
1. Create `upload_file_to_colab()` function for upload files `(train.json & test.json)` to `colab`.
2. Call function and upload files.

In [ ]:
# Step 1.
def upload_file_to_colab():
  uploaded = files.upload()
  for fileName in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes.'.format(name=fileName, length=len(uploaded[fileName])))

In [ ]:
# Step 2.
# upload train.json
print('Upload train.json please.\n')
upload_file_to_colab()

# upload test.json
print('Upload test.json please.\n')
upload_file_to_colab()

Upload train.json please.



Saving train.json to train.json
User uploaded file "train.json" with length 8695710 bytes.
Upload test.json please.



Saving test.json to test.json
User uploaded file "test.json" with length 4098518 bytes.


## 3. Data Pre-processing
1. Create a funciton `sentence_regex()` for do `Regular Expression`. When putting a sentence in this function, return a sentence that finished regular expression
2. Create `remove_empty_row()` function for remove all empty rows from dataframe.
3. Create `label_to_spam()` function for label data. If the value of column(`is_spam`) is `True`, return 1 else 0. 
4. Create a function `preprocessing_data()`. That is the main function of our pre-processing step. In this function, it will call all the above to process all sentences.
5. Call `preprocessing_data()` twice for both `test.json` and `train.json`.
6. Create dictionary objects for easy conversion to dataframe and remove empty rows.
7. Here, we just want to know the number of each label. (0 or 1)

In [ ]:
# Step 1.
def sentence_regex(sentence):
  complie = re.compile(u"[\u4e00-\u9fa5]+")

  sentence = sentence.replace("\n", ' ')
  sentence = re.sub(u'[a-zA-Z0-9]', '', sentence)

  sentence = re.findall(complie, sentence) #一則評論只取中文字，分成多行
  return ''.join(sentence)   

In [ ]:
# Step 2.
def remove_empty_row(df, col='text'):
  emptyTitleFIlter = (
        (df[col].isnull())|(df[col] == '')|(df[col] == '0')
        )
  df = df[~emptyTitleFIlter]
  return df

In [ ]:
# Step 3.
def label_to_spam(is_spam):
  if is_spam == True:
      return 1 
  return 0

In [ ]:
# step 4.
def preprocessing_data(filePath):
    data = pd.read_json(filePath)

    contents = list()
    spamLabels = list()
    length = len(data)

    # iterate each row, sentence(content) and label(is_spam), by index.
    for idx in range(length):
      sentence = data['content'][idx]
      is_spam = data['is_spam'][idx]

      # do regular expression
      # when finished, append it to a list (contents)
      sentence = sentence_regex(sentence)
      contents.append(sentence)
      
      # encode `is_spam` value.
      # when finished, append it to a list (spamLabels)
      is_spam = label_to_spam(is_spam)
      spamLabels.append(is_spam)

    return contents, spamLabels

In [ ]:
# Step 5.
train_contents, train_spamLabels = preprocessing_data('train.json')
print('* Train File:\n\tFirst sentence:', train_contents[0], '\n\tFirst label:', train_spamLabels[0])

test_contents, test_spamLabels = preprocessing_data('test.json')
print('* Test File:\n\tFirst sentence:', test_contents[0], '\n\tFirst label:', test_spamLabels[0])

* Train File:
	First sentence: 今天在亞太問到空機價是一萬九三月中上市要等嗎 
	First label: 1
* Test File:
	First sentence: 前陣子才購入心愛的珊瑚粉用到現在覺得還滿上手的借給朋友玩他們都驚呼速度很快順暢無比不會這不經讓我小小得意一下呵呵笑為了讓好好發揮最大效能衝一下因此安裝了許多遊戲工具導航照相的不過問題就由此而生啦出門在外使用這些電力一下就不足了有時候又找不到可以充電的地方沒電真的是很無奈囧開始想要添購行動電源的念頭搜尋一下發現有好多種牌子有些價錢便宜到誇張不敢下手希望是看起來有質感且大容量的行動電源不知道大家有什麼好的推薦嗎 
	First label: 1


In [ ]:
# Step 6 for train.json data
trainDict = {
    'text': train_contents,
    'label': train_spamLabels
}

trainDf = DataFrame(trainDict)
trainDf = remove_empty_row(trainDf, 'text')
trainDf.head()

,text,label
0,今天在亞太問到空機價是一萬九三月中上市要等嗎,1
1,之前在亞太的上看到即將上市的訊息後之後就都是謠言了門市人員也不確定上市時間讓人一直懸在那期待...,1
2,星期六的午後新竹南寮漁港旁雲水一方庭園餐廳銀河帶著他的筆記來拜訪薄薄的一本大大的面子亮亮的眼...,1
3,昨天去中華門市玩了一下手機想說趁最近資訊展的時候入手一支應該可以便宜一點不過玩了半天好像都沒...,1
4,體驗會時間年月日日開始入場首先感謝和舉辦這次的體驗會我很榮幸的能夠成為台中場的體驗者接下來我...,1


In [ ]:
# Step 6 for test.json data
testDict = {
    'text': test_contents,
    'label': test_spamLabels
}

testDf = DataFrame(testDict)
testDf = remove_empty_row(testDf, 'text')
testDf.head()

,text,label
0,前陣子才購入心愛的珊瑚粉用到現在覺得還滿上手的借給朋友玩他們都驚呼速度很快順暢無比不會這不經...,1
1,慕名已久的機皇風評一直都是很不錯正猶豫是否下手的時候就聽說要推出新的顏色所以又再延遲了一段時...,1
2,繼上次參加過新竹體驗會後這次是第二次參加三星的體驗會雖然地點在台北其實高鐵捷運還算方便還是不...,1
3,看到大家在砲轟早鳥禮才發現原來這麼多人都那麼看重贈品我覺得廠商給你再多再好的贈品最後還是會反...,1
4,有人跟我一樣在等的嗎來簽名報到一下吧順便看看這產品的人氣期待度微冷笑其實剛好明天休假,1


In [ ]:
# Step 7.
# define mask(filters)
train_true_filter = trainDf['label'] == 0
train_fake_filter = trainDf['label'] == 1
test_true_filter = testDf['label'] == 0
test_fake_filter = testDf['label'] == 1

true_train_count = len(trainDf.loc[train_true_filter])
true_test_count = len(testDf.loc[test_true_filter])
print('True Comment Count:', true_train_count + true_test_count)

fake_train_count = len(trainDf.loc[train_fake_filter])
fake_test_count = len(testDf.loc[test_fake_filter])
print('Fake Comment Count:', fake_train_count + fake_test_count)

True Comment Count: 15959
Fake Comment Count: 749


So far, the pre-processing of the text has been done. Before we do embedding, we need to merge two dataframe `trainDf` and `testDf`.
1. Create `concat_two_dataframe()` function.
2. Concate `trainDf` and `testDf`.
3. Because Fake's number less than real's, random sampling from True's data and then set the ratio is 3:1.
4. Call `concat_two_dataframe()` function again, as `dataset`, and this time we merge the data sampled. (`sampling_true_data` and `fake_data`)
5. Get all `text` and all `label` from dataset.

In [ ]:
# Step 1.
def concat_two_dataframe(df_1, df_2):
  concated = pd.concat([df_1, df_2], axis=0)
  concated = shuffle(concated)
  # reset index
  concated = concated.reset_index(drop=True)
  return concated

In [ ]:
# Step 2.1 concated train dataframe and test dataframe
concated_df = concat_two_dataframe(trainDf, testDf)

# Real and Fake Mask (filter)
fake_label_filter = concated_df['label'] == 1
true_label_filter = concated_df['label'] == 0

fake_data = concated_df.loc[fake_label_filter]
true_data = concated_df.loc[true_label_filter]

# Step 2.2 See the number of each label.
print('Real data total number:', len(true_data))
print('Fake data total number:', len(fake_data))

# Step 3.
# Because Fake's number less than real's.
# random sampling from True's data.
# set the ratio is 3:1
sampling_true_data = true_data.sample(
    n=len(fake_data) * 3,
    random_state=5487
  )

# Step 4.
dataset = concat_two_dataframe(sampling_true_data, fake_data)
dataset.info()

Real data total number: 15959
Fake data total number: 749
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996 entries, 0 to 2995
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2996 non-null   object
 1   label   2996 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 46.9+ KB


In [ ]:
# Step 5.
texts = dataset['text'].values
labels = dataset['label'].values

## 4. BERT Embedding
We can BERT embeddings now. In this tutorial, we load `bert-base-chinese`.
1. Instance a `tokenizer` object for encode each sentence.
2. Create `encode_sentence_by_bert()` for encoding every sentence place in this function.
3. Define a list(`input_id`) for storing all encoded tokens from BERT's tokenizer.
4. Pad sequences length to 100.
5. Create `MASK` list.
6. Convert data to tensor.
7. Create the `DataLoader` object for our training set.
8. Create BERT model and load BERT config.
9. Check GUP avalible.
10. Iterate batch data from our dataloader, and do embeddings.

In [ ]:
# Step 1.
PRETRAINED_MODEL_NAME = "bert-base-chinese"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

In [ ]:
# Step 2.
def encode_sentence_by_bert(sentence):
  encoded_sentence = tokenizer.encode(
        sentence, # Sentence to encode.
        add_special_tokens = True, # add [CLS] and [SEP]
        truncation = True,
        max_length = 100 
    )
  return encoded_sentence

In [ ]:
# Step 3.
input_ids = list()

# encode each sentence in texts.
for sentence in texts:
  encoded_sentence = encode_sentence_by_bert(sentence)
  # when sentence was encoded, append to input_ids list.
  input_ids.append(encoded_sentence)

print('Number of sentence:', len(input_ids))
print('Length of encode sentence:', len(input_ids[0]))

Number of sentence: 2996
Length of encode sentence: 83


In [ ]:
# Step 4.
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 100
input_ids = pad_sequences(
    input_ids, 
    maxlen= MAX_LEN, 
    dtype="long", 
    value=0, # padding by value 0
    truncating="post", # truncating from behind
    padding="post") # padding from behind

In [ ]:
# Step 5.
attention_masks = list()
for sentence in input_ids:
    att_mask = [int(token_id > 0) for token_id in sentence]
    attention_masks.append(att_mask)

In [ ]:
# Step 6.
inputs = torch.tensor(input_ids)
labels = torch.tensor(labels)
masks = torch.tensor(attention_masks)

In [ ]:
# Step 7. Create the DataLoader for our training set.
batch_size = 8
train_dataset = TensorDataset(inputs, masks, labels)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

In [ ]:
# Step 8. 
# Create BERT model
'''
The bare Bert Model transformer outputting raw hidden-states without any specific head on top.
'''
config = BertConfig.from_pretrained(PRETRAINED_MODEL_NAME, output_hidden_states=True)
model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME, config=config)
model.cuda()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# Step 9. 
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# Step 10.
train_label = list()
train_set = list()

count=0
for step, batch in enumerate(train_dataloader):
  batch_tensor = tuple(tensr.to(device) for tensr in batch)
  # b_ means batch
  b_input_ids, b_input_mask, b_labels = batch_tensor
  outputs = model(b_input_ids, b_input_mask)

  # get last_layer 
  last_layer = outputs[0] # last layer
  label_ids = b_labels.to('cpu').numpy() 
  train_label.append(np.array(label_ids)) # real label for training a SVM model.

  for state in last_layer:
    # state size: (100, 768)
    allWordVecs = list()
    for word_unit in state:
      # size (768)
      allWordVecs.append(word_unit.detach().cpu().numpy())
    train_set.append(allWordVecs)

In [ ]:
labels = list()
for i in train_label:
    for j in i:
        labels.append(j)
labels = np.array(labels)
print('Length of Labels:', len(labels))

Length of Labels: 2996


In [ ]:
# Show the shape of train_set.
layer = np.array(train_set)
train_set = None
print(layer.shape)

(2996, 100, 768)


## 5. Build SVM Classifier

In [ ]:
# split dataset 
X_train, X_test, y_train, y_test = \
    train_test_split(
        layer, 
        labels,
        test_size=0.3,
        random_state=0,
        stratify=labels
    )

# first feature CLS
X_train = X_train[:, 0, :]
X_test = X_test[:, 0, :]
print(X_train.shape)

model_svm = SVC(kernel='rbf')
model_svm.fit(X_train, y_train)
prediction = model_svm.predict(X_test)
accuracy = metrics.accuracy_score(y_test, prediction.round())

(2097, 768)


In [ ]:
accuracy

0.8832035595105673